In [3]:
import pandas as pd
import ast
from collections import Counter
import re

In [2]:
#df_coutnries = df[df['countries'] != '<UNK>']
#df_coutnries.to_csv('NLGRecipe_KownCountriesOnly.csv', index=False)

In [4]:
df = pd.read_csv('NLGRecipe_KownCountriesAndAuthenticity.csv')
print(len(df))

147000


# Take top 50 countries and take rows only from those countries

In [4]:
#Top countries
series = df['countries'].value_counts()

In [ ]:
series[50:80]

## Take the top 50 NER list ingredients -- creating the stopword list

In [6]:
def get_top_ingredients(df, country='France'):
    df_country = df[df['countries'] == country]
    ingr_list = list(df_turk['NER'])
    new_ingr_list = []
    for str_ingr in ingr_list:
        mew_ingr_list.extend(ast.literal_eval(str_ingr))
    sorted_dict = Counter(new_ingr_list)
    return sorted_dict

## For each data frame -- do the processing of ingredient similarity (with dissimilarity constraint)

In [7]:
def cleaning_lists(raw_list, stopword_list):

    #Cleaning list from the raw dataset
    new_ingr_list = []
    for str_ingr in raw_list:
        temp = ast.literal_eval(str_ingr)
        #Removing stopwords from list
        temp2 = [el for el in temp if el not in stopword_list]
        new_ingr_list.append(temp2)
    
    return new_ingr_list

In [8]:
def recipe_suggestion(ingredient_list, top=5, lambda_=0.2):

    #List of ingredient from already selected indexes taht will be penalized
    used_ingredient = []
    
    #Receive a list of recipes per country and give the 5 best recipe
    avg_list = []
    kept_recipes = []
    for n in range(top):
        #print('used_ingredient : ', used_ingredient)
        max_score = 0.
        for i, ingredients in enumerate(ingredient_list):
            #print(ingredients)
            temp_list = ingredient_list[:i] + ingredient_list[i+1:]
            #print(len(temp_list))
            #print(temp_list[0:3])
            #Getting the average number of common elements with all other items
            if n == 0:                    ## At first iterations we need to estimate number of common element sbetween items but we don't want to make the calculation every steps -- efficiency
                #print('prout')
                if len(ingredients) == 0:
                    avg_sim = 0.
                else:
                    nb_common = 0
                    for elem in temp_list:
                        nb_common += (len(list(set(ingredients).intersection(elem))))#/len(ingredients))
                    avg_sim = nb_common / len(temp_list)
                avg_list.append(avg_sim)
                #print('avg : ', avg_sim)
            else:                       #Getting the common number of ingredients with the alredy used ones
                #Calculating the penalization score -- we count the total number of time each ingredient appears in the already used list
                if len(ingredients) == 0:
                    used_score = 0.
                else:
                    #print('ingredient_list: ', ingredients)
                    #print('used_ingredient: ', used_ingredient)
                    used_score = sum(el in ingredients for el in used_ingredient) / len(used_ingredient)
                    #print('used_score: ',used_score)
                avg_sim = (1-lambda_)*avg_list[i] - lambda_ * used_score
                #print('avg_sim: ',avg_sim)
                
            #Saving the index of the highest similarity recipe
            if avg_sim > max_score:
                max_score = avg_sim
                index_max = i
                #print('maxes: ',max_score, index_max)
                
        #print('================================= ', avg_sim, index_max, max_score)
        #After calculating all score we append the ingredients to the used ones
        #ingredient_list
        used_ingredient.extend(ingredient_list[index_max])
        
        #We keep the index of the recipe in the list
        kept_recipes.append(index_max)
    
    return kept_recipes

In [9]:
def getting_recipe_titles(df, stopword_list, country_name='Italy'):

    df_country = df[df['countries'] == country_name]
    print(country_name, ": ", len(df_country))
    title_list = list(df_country['title'])
    entity_list = list(df_country['NER'])

    clean_entities = cleaning_lists(entity_list, stopword_list)
    #print(clean_entities)
    recipe_indexes = recipe_suggestion(clean_entities)

    final_title_list = []
    for index in recipe_indexes:
        #break
        final_title_list.append(title_list[index])
    
    return final_title_list

# Runing the code for the selected country list

In [10]:
stopword_ingredients = ['sugar', 'milk', 'salt', 'water', 'pepper', 'butter', 'oil', 'olive oil', 'vegetable oil', 'garlic', 'onion', 
                 'eggs', 'flour', 'brown sugar', 'vinegar', 'powdered sugar'] #To Add Flour Eggs

In [ ]:
fileInputName = "Selected_coutrnies_forRecipies.txt"
fileInputOpen = open(fileInputName, "r")
message = fileInputOpen.read()

with open(fileInputName, "r") as f:
    country_raw = f.readlines()

df_suggest = pd.DataFrame(columns=['Country', 'Recipe_1', 'Recipe_2', 'Recipe_3', 'Recipe_4', 'Recipe_5'])

country_list = []
for country in country_raw:
    country = re.sub("\n", "", country)
    recipe_list = getting_recipe_titles(df, stopword_ingredients, country_name=country)
    dict_temp = {'Country': country, 'Recipe_1':recipe_list[0], 'Recipe_2':recipe_list[1], 'Recipe_3':recipe_list[2], 'Recipe_4':recipe_list[3], 'Recipe_5':recipe_list[4]}
    print(dict_temp)
    df_suggest = df_suggest._append(dict_temp, ignore_index = True)
df_suggest.to_csv('Suggested_Recipe_LowDiversity_2.csv', index=False)

# Suggestion with authentic Recipes - from countries

In [5]:
df_auth = df[df['authenticity'] == 'authentic']

In [13]:
df_auth_save = df_auth.drop(columns=['Unnamed: 0', 'link', 'source', 'ingredients', 'directions'])
df_auth_save.head()

,title,NER,countries,authenticity
186,Lithuanian Sweet Bread(Pyragas; A Simplified V...,"[""flour"", ""sugar"", ""active dry yeast"", ""milk"",...",Lithuania,authentic
1177,The Original Mexican Cornbread,"[""sour cream"", ""meal"", ""eggs"", ""cream-style co...",Mexico,authentic
1362,Authentic Swedish Meat Balls(Great For Parties),"[""ground pork"", ""onion"", ""ground allspice"", ""b...",Åland Islands,authentic
3418,Authentic French Dressing(For Bibb And Leaf Le...,"[""mustard"", ""garlic"", ""wine vinegar"", ""oil""]",France,authentic
3680,Original German Sweet Chocolate Cake,"[""chocolate"", ""boiling water"", ""butter"", ""suga...",Germany,authentic


In [14]:
df_auth_save.to_csv('AuthenticSuggestions.csv', index=False)

In [15]:
len(df_auth_save)

839

In [4]:
### Statistics for the authentic countries
df_auth = pd.read_csv('AuthenticSuggestions.csv')

In [8]:
country_series = df_auth['countries'].value_counts()

In [9]:
country_series.to_csv('AuthCountries.csv')

# Tests

In [5]:
import pandas as pd
import ast
from collections import Counter
import re

In [33]:
df = pd.read_csv('Full_NLGRecipe_Authenticity.csv')

In [35]:
df.head()

,Unnamed: 0,title,ingredients,directions,link,source,NER,authenticity
0,0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,Gathered,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu...",variation
1,1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,Gathered,"[""beef"", ""chicken breasts"", ""cream of mushroom...",variation
2,2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,Gathered,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar...",variation
3,3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,Gathered,"[""chicken"", ""chicken gravy"", ""cream of mushroo...",variation
4,4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,Gathered,"[""peanut butter"", ""graham cracker crumbs"", ""bu...",variation


In [37]:
df_auth = df[df['authenticity'] == 'authentic']

In [43]:
df_auth.to_csv('All_authentic_recipes.csv', index=False)